# Change tensorflow version from 2.6.4 to 2.9.1

I noticed that the default version of tensorflow (2.6.4) has a problem of printing unecessary clean up messages. for this reason I upgraded to tensorflow 2.9.1

In [ ]:
!pip uninstall tensorflow --yes
!pip install tensorflow_decision_forests
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 --yes
!pip install kerassurgeon

# Library
here are the required packages/modules we will use to perform the tasks

In [ ]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import glob
import pathlib
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import datasets, layers, models,Input,Model 
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import  Bidirectional, Conv2D, BatchNormalization, MaxPooling2D, Flatten, LSTM, Dense, Lambda, Dropout,Reshape
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tensorflow.keras.metrics import Accuracy, Recall,Precision
from sklearn.tree import DecisionTreeClassifier as Decisiontree
from sklearn.svm import SVC as Supportvectorclassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
from functools import reduce
from kerassurgeon.operations import delete_layer, insert_layer
from keras.utils import to_categorical
import pickle

Just checking if tensorflow was sucessfully upgraded.

In [ ]:
print(tf.__version__)

# Introduction
This study utilized a secondary radiography dataset, retrieved from the Kaggle data repository. The original copy was compiled from publicly available posterior-to-anterior (AP) chest x-rays by a group of research collaborators from Qatar and Bangladesh. The experimental setting involved a placebo type of design with 3 study groups; these include A control group with 423 x-ray images from normal people, an experimental group with 1579 x-ray images from COVID infected persons, and a group with viral pneumonia infected persons with 1485 images (Chowdhury et al., 2020). The data will be split into a training set with 75% and a testing set with 25%, during the preprocessing step. The testing set will be held to help evaluate the performance of the models on new data.

# Loading the data

Here I will load the image data.I will use the tensorlow `image_from_directory()`  which I consider efficient in loading image data.  Images will be loaded in their original form of `299x 299` shape.

We have a total of **13808** images in consideration, **10192** are from normal cases while **3616** are from covid infected cases. The overall data was randomly split into **80%(11047)** training images and **20%(2761)** testing images.

About **8163** of the training images are from normal people while the rest are from covid patients. For testing **2029** are from normal people while the rest are covid infected cases. 

The data is saved as  batched tensorflow images with batch sizes of 32 images.There is a total of 346 batches on the training set and 87 images on the training.

Data link:https://www.kaggle.com/najwa2030/the-original-model/edit

In [ ]:
# some parameter settings

batch_size=32
image_height=299
image_width=299
n=1000
epochs=100

here we are loading the images from the folder they are saved.Tensorflow pipeline will be used to allow flow of images from directory.

In [ ]:
# reading images from directory(folder)

data_dir= "../input/mycoviddata/covid images/binary"
train_ds= tf.keras.utils.image_dataset_from_directory(
  data_dir,
  color_mode='grayscale',
  labels='inferred',
  label_mode='categorical',
  subset="training",
  image_size=(image_height,image_width),
  validation_split=0.20,
  seed=100)

test_ds= tf.keras.utils.image_dataset_from_directory(
  data_dir,
  color_mode='grayscale',
  labels='inferred',
  label_mode='categorical',
  subset="validation",
  image_size=(299, 299),
  validation_split=0.20,
  seed=100)

Your images belong to these two directories. COVID and Normal

In [ ]:
# Class names
train_ds.class_names

Here we collected details of the image, such as image counts in each group, training size and testing size etc. they are the details graphed below.

In [ ]:
# preparing summary counts
data_dir=pathlib.Path('../input/mycoviddata/covid images/binary')

# function to get labs 
def fetch_labels(filepath):
    return str(filepath).split('/')[-2]

# Training labels
labs=list(map(fetch_labels,train_ds.file_paths))
trainlabs=pd.DataFrame(labs,columns=["Labels"])

# testing labels
labs=list(map(fetch_labels,test_ds.file_paths))
testlabs=pd.DataFrame(labs,columns=["Labels"])

# Overall image count
df=pd.DataFrame([["Normal",len(list(data_dir.glob('Normal/*.png')))],
              ["COVID",len(list(data_dir.glob('COVID/*.png')))]]
             , columns=['Label','count'])
# batch count
df1=pd.DataFrame([["Training",len(train_ds)],
              ["Testing",len(test_ds)]]
             , columns=['Label','count'])

In [ ]:
print("-------------- Overall data ------------------------------------\n")
print("image count:",len(list(data_dir.glob('*/*.png'))))
print("Normal image count:",len(list(data_dir.glob('Normal/*.png'))))
print("Covid image count:",len(list(data_dir.glob('COVID/*.png'))))

print("\n------------Train test split--------------------------------\n")
print("Training count:",len(trainlabs))
print("Testing count:",len(testlabs))
print("Number of batches training set:",len(train_ds))
print("Number of batches testing set:",len(test_ds))

In [ ]:
figure,ax=plt.subplots(nrows=2,ncols=2,figsize=(12,10))
sns.barplot(x = 'Label',y = 'count',data = df,ax=ax[0,0])
sns.countplot(x='Labels',data=trainlabs,ax=ax[0,1])
sns.countplot(x='Labels',data=testlabs,ax=ax[1,0])
sns.barplot(x = 'Label',y = 'count',data = df1,ax=ax[1,1])
ax[0,0].set_title('Total Images')
ax[0,1].set_title('Training images')
ax[1,0].set_title('Testing images')
ax[1,1].set_title('Batch counts')
for p, label in zip(ax[0,1].patches, trainlabs['Labels'].value_counts().index):
    ax[0,1].annotate(p.get_height(), (p.get_x()+0.25, p.get_height()+20))
for p, label in zip(ax[1,0].patches, testlabs['Labels'].value_counts().index):
    ax[1,0].annotate(p.get_height(), (p.get_x()+0.25, p.get_height()+20))
for p, label in zip(ax[0,0].patches, df['count'].index):
    ax[0,0].annotate(round(p.get_height()), (p.get_x()+0.25, p.get_height()+20))
for p, label in zip(ax[1,1].patches, df1['count'].index):
    ax[1,1].annotate(round(p.get_height()), (p.get_x()+0.25, p.get_height()+5))
ax[0,0].set_xlabel("")
ax[0,1].set_xlabel("")
ax[1,0].set_xlabel("")
ax[1,1].set_xlabel("")
    
plt.savefig('original data1.png')

# Visualizing a sample of the images

Below I visualized a sample of 16 images,to see if the labels are matched properly.

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(16):
        ls=labels[i].numpy()
        x=[j for j, y in enumerate(ls) if y == 1]
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"),cmap='Greys_r')
        plt.title(train_ds.class_names[x[0]])
        plt.axis("off")
        
plt.savefig('original data2.png')

Here are tensorflow commands to allow prefetching and caching. This has to do with tensorflow, when using tensorflow caching means, the data will cached after first epoch. so will not be loaded afresh everytime. prefetching means processing is overlapped(sort of multi-tasking)

In [ ]:
# performance configuration
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def benchmark(dataset):
    num_epochs
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    print("Execution time:", time.perf_counter() - start_time)


# Modelling 

 Over the recent past, Convolution Neural Networks (CNN or Convnet) has gained popularity in visual imagery and video processing (Wu et al., 2017). This is due to its architecture which allows zooming into images to get pixel information and pooling ability. Artificial Neural network (ANN) on the other hand uses weighting and activation functions to perform regression/classification tasks. This study will make use of sequential layers of CNN to perform feature extraction and classification. The study architecture is inspired by a VGG16 architecture and will involve a **feature extraction layers** and **classification layers**. The feature extraction part includes sequences of convolution and pooling layer. Conversely, the classification part includes two layers of feed-forward artificial neural networks (ANN), tasked with classifying images based on features extracted during feature extraction.  The model will be implemented using Keras.

![image.png](attachment:0129c5d2-bc8a-4f5d-999f-41829a5cb39a.png)

Here is our model, This section is the feature extraction part, each block  contains convolution layers normalization layers and maximum pooling layers.

In [ ]:
model=Sequential(name="Full_Model")
# Block 1
model.add(Input(shape=(image_height,image_width,1),name="input"))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu',name="block1_conv_1"))
model.add(BatchNormalization(name="block1_batch_normalization1"))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu',name="block1_conv_2"))
model.add(BatchNormalization(name="block1_batch_normalization2"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block1_maxpool"))

# Block 2

model.add(Conv2D(128, (3, 3), padding='same', activation='relu',name="block2_conv_1"))
model.add(BatchNormalization(name="block2_batch_normalization1"))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu',name="block2_conv_2"))
model.add(BatchNormalization(name="block2_batch_normalization2"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block2_maxpool"))
          
# Block 3
model.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_1"))
model.add(BatchNormalization(name="block3_batch_normalization1"))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_2"))
model.add(BatchNormalization(name="block3_batch_normalization2"))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_3"))
model.add(BatchNormalization(name="block3_batch_normalization3"))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),name="block3_maxpool"))

# Block 4
model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_1"))
model.add(BatchNormalization(name="block4_batch_normalization1"))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_2"))
model.add(BatchNormalization(name="block4_batch_normalization2"))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_3"))
model.add(BatchNormalization(name="block4_batch_normalization3"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block4_maxpool"))

# fifth convolution layer

model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_1"))
model.add(BatchNormalization(name="block5_batch_normalization1"))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_2"))
model.add(BatchNormalization(name="block5_batch_normalization2"))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_3"))
model.add(BatchNormalization(name="block5_batch_normalization3"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block5_maxpool"))
#  flatten
model.add(Flatten(name="flatten_layer"))

## Adding dense layers

Here we will connect the feature extraction part with the ANN classifier to produce a single model like shown on the figure above.

In [ ]:
# Dense connected layers
model.add(Dense(units=64,activation="relu"))
model.add(Dense(units=64,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
#model.summary()

# Compile and fit the ANN model. 

A sparse categoricalCrossentropy will be used.

In [ ]:
model.compile(
  optimizer='adam',
  loss="categorical_crossentropy",
  metrics=["accuracy",
           tf.keras.metrics.Recall(name="Sensitivity",class_id=0),
           tf.keras.metrics.Recall(name="Specificity",class_id=1),
           tf.keras.metrics.Precision(name="Precision",class_id=0),
          tfa.metrics.F1Score(num_classes=2, average="micro")])

To make the code we will use the first 20 Batches. But later we will run the code with the full data

In [ ]:
train_ds=train_ds.take(n)

# fitting the model

In [ ]:
hist=model.fit(
 train_ds,
  validation_data=test_ds,
 epochs=epochs)


In [ ]:

#Accuracy
figure,ax=plt.subplots(nrows=1,ncols=2,figsize=(15,6))
ax[0].plot(hist.history['accuracy'])
ax[0].plot(hist.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')

# loss
ax[1].plot(hist.history['loss'])
ax[1].plot(hist.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper left')

Graphing accuracy on training vs testing set.

In [ ]:
#Accuracy
figure,ax=plt.subplots(nrows=1,ncols=2,figsize=(15,6))
ax[0].plot(hist.history['accuracy'])
ax[0].plot(hist.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')

# loss
ax[1].plot(hist.history['loss'])
ax[1].plot(hist.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper left')
plt.savefig('original data3.png')

# Here is the model perfomance metric.

In [ ]:
#m1=model.evaluate(test_ds)
st = time.time()
m1=model.evaluate(test_ds)[1:6]
et = time.time()
elapsed_time=round((et - st)/len(testlabs),4)
print('Execution time:', elapsed_time, 'seconds')
m1.append(elapsed_time)
mod1=pd.DataFrame({"Measure":['Accuracy','Sensitivity','Specificty','Precision',"F1-score","Excecution time"],
    "Original Dataset":[np.round(float(i), 4) for i in m1]})
mod1

# save model and data

In [ ]:
model.save("my_model")
train_ds.save("train_ds")
test_ds.save("test_ds")


with open("original_hist.pkl","wb") as file:
    pickle.dump(hist,file)
    
#with open("original_hist.pkl","rb") as file:
#    hist=pickle.load(file)
    
#model = keras.models.load_model('my_model')
#train_ds=tf.data.Dataset.load("train_ds")
#test_ds=tf.data.Dataset.load("test_ds")